# Creating FABnet Networks

One of the primary features of FABRIC is its ability to deploy distributed sets of VMs connected with high-performance wide-area networks. FABRIC provides two WAN services at two levels, layer 2 circuits and layer 3 IP connectivity.  Both types of networks use FABRIC’s high-performance network infrastructure.  

Some FABRIC experiments require the lower layer 2 circuits.  These experiments tend to involve users needing to configure routers/switches in the core of the network.  However, many experiments are able to use standard IP (layer 3) services.  These experiments tend to involve connecting applications or data management services across FABRIC’s high-performance wide-area networks.  

This tutorial walks the user through creating and configuring experiments using FABRIC’s FABnetv4 (IPv4) networking services. During this tutorial users will create FABnet networks and configure the subnets, IP addresses, and routes for several common usecases. 

After completing this tutorial, users will be able to: 

- Create FABnet networks
- Configure routes on nodes to use FABnet networks with a slice
- Configure routes on nodes to use FABnet networks between slices
- Selectively configure routes on nodes to allow/prevent communication between specific slices

## FABnetv4 and FABnetv6

FABRIC provides a pair of layer 3 IP networking services across every FABRIC site.  FABnetv4 (IPv4) and FABnetv6 (IPv6). You can think of these services as pair of private internets that connect  FABRIC sites using its high-performance network links.  Users can create private subnets that can route IP traffic through the shared FABnet networks.

The FABlib API enables FABRIC users to create isolated FABnet networks.  Each FABnet network is issued a subnet of the greater FABnet internets.  Individual VMs can be added to any FABnet networks colocated with he VM. 

Upon creation, your individual FABnet network is issued a subnet and gateway by the FABRIC control framework. The user is responsible for configuring the subnet, routes, and IP addresses for their slice and can do so in any way that they wish.  FABRIC is responsible for forwarding traffic between all FABnet networks on behalf of the users.



## Imports

Import the FABlib library and create a FABlib manager. For this turtorial you will also need to import some Python IP handling packages.

In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()             
config = fablib.show_config()


# Exercise 1 (Beginner): FABnet Networks


<img src="./figs/fabnet_2_sites.png" width="80%"><br>

## Create the Experiment Slice


The following cells creates two nodes, on different sites, with basic NICs connected to FABRIC's FABnetv4 internet.  This slice will be (re)used by all exercises in this tutorial.

Running the cell creates two node with one NIC component is added to each node.  This example uses components of model `NIC_Basic` which are SR-IOV Virtual Function on a 100 Gpbs Mellanox ConnectX-6 PCI device.  The `get_interfaces` method returns a list of all interfaces associated with the NIC.  Since a `NIC_Basic` has only one interface, you will find only one interface in the list. 

Next, add a separate `l3network` for each site and pass the list of interfaces on that site that you want to connect to FABnetv4. All interfaces passed to `l3network` must be on the same site and each network will be placed on that site.  By default, a node is put on a random site.  If you want to ensure that your nodes are all on different sites you can specify the name of the sites in the `add_node` methods.  You can use the `fablib.get_random_site()` method to get a set of random site names that guarantee that the sites are different. 


In [ ]:
slice_name = 'MySlice'
[site1,site2] = fablib.get_random_sites(count=2)
print(f"Sites: {site1}, {site2}")

node1_name = 'Node1'
node2_name = 'Node2'

network1_name='net1'
network2_name='net2'

node1_nic_name = 'nic1'
node2_nic_name = 'nic2'

In [ ]:
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)
    
    # Node1
    node1 = slice.add_node(name=node1_name, site=site1)
    iface1 = node1.add_component(model='NIC_Basic', name=node1_nic_name).get_interfaces()[0]
    
    # Node2
    node2 = slice.add_node(name=node2_name, site=site2)
    iface2 = node2.add_component(model='NIC_Basic', name=node2_nic_name).get_interfaces()[0]
    
    # NetworkS
    net1 = slice.add_l3network(name=network1_name, interfaces=[iface1], type='IPv4')
    net2 = slice.add_l3network(name=network2_name, interfaces=[iface2], type='IPv4')
    
    #Submit Slice Request
    slice.submit()
except Exception as e:
    print(f"Exception: {e}")

## Configure IP Addresses


Now you need to configure IP addresses on their new nodes.  FABlib provides some useful methods to help you configure basic IP addresses. 

### Get the Assigned Subnets

FABnetv4 networks are assigned a subnet and gateway by FABRIC.  You can get the subnet and available IPs from the FABlib objects. 

In [ ]:
try:
    network1 = slice.get_network(name=network1_name)
    network1.show()

    network2 = slice.get_network(name=network2_name)
    network2.show()
except Exception as e:
    print(f"Exception: {e}")

### Get Configuration Variables

Get and print the values needed for the configuration below.


<img src="./figs/fabnet_routes.png" width="80%"><br>


In [ ]:
try:
    network1_gateway = network1.get_gateway()
    network1_subnet = network1.get_subnet()

    network2_gateway = network2.get_gateway()
    network2_subnet = network2.get_subnet()
    
    node1 = slice.get_node(name=node1_name)        
    node1_iface = node1.get_interface(network_name=network1_name)  
    node1_dev = node1_iface.get_os_interface()
    node1_ip_addr = network1.get_available_ips()[0]
    
    node2 = slice.get_node(name=node2_name)        
    node2_iface = node2.get_interface(network_name=network2_name)  
    node2_dev = node2_iface.get_os_interface()
    node2_ip_addr = network2.get_available_ips()[0]

    
    print(f"Network1:")
    print(f"network1_gateway: {network1_gateway}")
    print(f"network1_subnet: {network1_subnet}")

    print(f"\nNetwork2:")
    print(f"network2_gateway: {network2_gateway}")
    print(f"network2_subnet: {network2_subnet}")

    print(f"\nNode1:")
    print(f"node1_ip_addr: {node1_ip_addr}")
    print(f"node1_dev: {node1_dev}")

    print(f"\nNode2:")
    print(f"node2_ip_addr: {node2_ip_addr}")
    print(f"node2_dev: {node2_dev}")

except Exception as e:
    print(f"Exception: {e}")

### Configure Node1

Get the node and the interface you wish to configure.  You can use `node.get_interface` to get the interface that is connected to the specified network.  Then get an IP address from the list of available IPs and call `iface.ip_addr_add` to set the IP and subnet.  

Then set a route from *this network* to the *other network* through the specified gateway.


Optionally, use the `node.execute()` method to show the results of adding the IP address and route.



In [ ]:
try:
    node1 = slice.get_node(name=node1_name)        
    node1_iface = node1.get_interface(network_name=network1_name)  
    
    print(f"Assigning IP {node1_ip_addr} to {node1.get_name()} dev {node1_dev}") 
    node1_iface.ip_addr_add(addr=node1_ip_addr, subnet=network1_subnet)
    
    print (f"\nNetwork settings for dev {node1_dev}:")
    stdout, stderr = node1.execute(f'ip addr show {node1_dev}')
    
    print(f"Adding route, subnet: {network2_subnet}, gateway: {network1_gateway}") 
    node1.ip_route_add(subnet=network2_subnet, gateway=network1_gateway)
    
    print (f"{node1.get_name()} Routes:")
    stdout, stderr = node1.execute(f'ip route list')
except Exception as e:
    print(f"Exception: {e}")

### Configure Node2

Repeat the steps to add the next available IP to the second node and a route to the first network.

In [ ]:
try:
    node2 = slice.get_node(name=node2_name)        
    node2_iface = node2.get_interface(network_name=network2_name)  
    
    print(f"Assigning IP {node2_ip_addr} to {node2.get_name()} dev {node2_dev}") 
    node2_iface.ip_addr_add(addr=node2_ip_addr, subnet=network2_subnet)
    
    print (f"\nNetwork settings for dev {node2_iface.get_os_interface()}:")
    stdout, stderr = node2.execute(f'ip addr show {node2_dev}')
    
    print(f"Adding route, subnet: {network1_subnet}, gateway: {network2_gateway}") 
    node2.ip_route_add(subnet=network1_subnet, gateway=network2_gateway)
    
    print (f"{node2.get_name()} Routes:")
    stdout, stderr = node2.execute(f'ip route list')
except Exception as e:
    print(f"Exception: {e}")

## Test the connection

- `ping`: Tests connectivity and latency
- `tracepath`: Shows hops through each router


In [ ]:
try:
    node1 = slice.get_node(name=node1_name)        

    stdout, stderr = node1.execute(f'ping -c 5 {node2_ip_addr}')
    stdout, stderr = node1.execute(f'tracepath {node2_ip_addr}')
except Exception as e:
    print(f"Exception: {e}")

## Questions:
    
- Is this the expected RTT?
- Why are some of the tracepath entries missing?
- What would happen if you set the VMs default route to the FABnet gateway? 

## Clean up for the next Exercise

Remove the routes you added to your VMs before moving on to the next level.

In [ ]:
try:    
    print(f"Clean up node1")
    node1 = slice.get_node(name=node1_name)        
    node1_iface = node1.get_interface(network_name=network1_name)  
    
    print(f"Removing route, subnet: {network2_subnet}, gateway: {network1_gateway}") 
    node1.ip_route_del(subnet=network2_subnet, gateway=network1_gateway)
    
    print (f"{node1.get_name()} Routes:")
    stdout, stderr = node1.execute(f'ip route list')
    
except Exception as e:
    print(f"Exception: {e}")
    
try:    
    print(f"Clean up node2")
    node2 = slice.get_node(name=node2_name)        
    node2_iface = node2.get_interface(network_name=network2_name)  
    
    print(f"Removing route, subnet: {network1_subnet}, gateway: {network2_gateway}") 
    node2.ip_route_del(subnet=network1_subnet, gateway=network2_gateway)
    
    print (f"{node2.get_name()} Routes:")
    stdout, stderr = node2.execute(f'ip route list')
    
except Exception as e:
    print(f"Exception: {e}")

# Exercise 2 (Intermediate): Enable Connections to your Neighbor's Nodes

Like the public Internet, the FABnet routers forward traffic on behalf of all FABRIC slices.  In the previous exercise, you configured your VMs with routes such that only traffic destined to subnets within your slice are routed to FABnet.  For many experiments, only setting routes to/from your slice is adequate.  However, you can also use FABnet to send traffic to/from other slices.  Communication between slices using FABnet can be accomplished by including additional routes in your VMs. In this exercise you will work with a partner and send traffic to/from each other's VMs.

The easiest way to set up routes between two slices is to set one route that matches all possible subnets on FABnet.  In FABnetv4, all subnets issued to users come from `10.128.0.0/10`.  In order to send traffic between your VMs and your partner's VMs, you both can set routes in your VMs to `10.128.0.0/10` via the gateway assigned to your respective networks.

<img src="./figs/fabnet_route_all.png" width="80%"><br>


## Set the FABnet Subnet




In [ ]:
fabnetv4_subnet = IPv4Network('10.128.0.0/10')

## Add a route in Node1 to all of FABnet subnet via the Gateway

In [ ]:
try:    
    node1 = slice.get_node(name=node1_name)        
    node1_iface = node1.get_interface(network_name=network1_name)  
     
    print(f"Adding route, subnet: {fabnetv4_subnet}, gateway: {network1_gateway}") 
    node1.ip_route_add(subnet=fabnetv4_subnet, gateway=network1_gateway)

    print (f"{node1.get_name()} Routes:")
    stdout, stderr = node1.execute(f'ip route list')
except Exception as e:
    print(f"Exception: {e}")

## Add a route in Node2 to all of FABnet subnet via the Gateway

In [ ]:
try:    
    node2 = slice.get_node(name=node2_name)        
    node2_iface = node2.get_interface(network_name=network2_name)  
        
    print(f"Adding route, subnet: {fabnetv4_subnet}, gateway: {network2_gateway}") 
    node2.ip_route_add(subnet=fabnetv4_subnet, gateway=network2_gateway)
    
    print (f"{node2.get_name()} Routes:")
    stdout, stderr = node2.execute(f'ip route list')
except Exception as e:
    print(f"Exception: {e}")

## Test the route between your VMs

In [ ]:
try:
    node1 = slice.get_node(name=node1_name)        

    stdout, stderr = node1.execute(f'ping -c 5 {node2_ip_addr}')
    stdout, stderr = node1.execute(f'tracepath {node2_ip_addr}')
except Exception as e:
    print(f"Exception: {e}")

## Test the route between your VMs and your Partner's VMS

Say "Hi" to your neighbor and exchange the IP addresses that you have each assigned to your respective VMs.  

Set the `partner_vm_ip` in the following cell to the IP address of your partner's VM. Test the connection with `ping` and `tracepath`.


In [ ]:
try:
    partner_vm_ip = '<PARTNER_VM_IP>'
    
    node1 = slice.get_node(name=node1_name)        

    stdout, stderr = node1.execute(f'ping -c 5 {partner_vm_ip}')
    stdout, stderr = node1.execute(f'tracepath {partner_vm_ip}')
except Exception as e:
    print(f"Exception: {e}")

## Questions:
    
- What could you use this capability for?
- Try providing a service to your partner's VM from your own.

## Clean up for the next Exercise 

Remove the routes you added to your VMs before moving on to the next level.

In [ ]:
try:    
    print(f"Clean up node1")
    node1 = slice.get_node(name=node1_name)        
    node1_iface = node1.get_interface(network_name=network1_name)  
    
    print(f"Removing route, subnet: {fabnetv4_subnet}, gateway: {network1_gateway}") 
    node1.ip_route_del(subnet=fabnetv4_subnet, gateway=network1_gateway)
    
    print (f"{node1.get_name()} Routes:")
    stdout, stderr = node1.execute(f'ip route list')
    
except Exception as e:
    print(f"Exception: {e}")
    
try:    
    print(f"Clean up node2")
    node2 = slice.get_node(name=node2_name)        
    node2_iface = node2.get_interface(network_name=network2_name)  
    
    print(f"Removing route, subnet: {fabnetv4_subnet}, gateway: {network2_gateway}") 
    node2.ip_route_del(subnet=fabnetv4_subnet, gateway=network1_gateway)
    
    print (f"{node2.get_name()} Routes:")
    stdout, stderr = node2.execute(f'ip route list')
    
except Exception as e:
    print(f"Exception: {e}")

# Exercise 3 (Advanced): Use Specific Routes to your Neighbor's Nodes

Although it is easier to set a general route to all possible FABnet subnets, doing so enables any FABRIC VM to connect to your VM. Often you will want to prevent most other FABRIC slices from accessing your VMs.  

In this exercise, use the `ip_route_del` and `ip_route_add` method that you have learned about to selectively add routes to your VMs such that only other specific subnets can access your VMs.  

Try the following:

- Allow connection to multiple partners without allowing access to all other slices.
- Allow connection to your partner from some of your VMs but not all of them.
- Allow connections from some of your partner's VMs but not all of them.

<img src="./figs/fabnet_specific_routes.png" width="80%"><br>





In [ ]:
partner_vm_ip = '1.2.3.4'
partner_subnet = IPv4Network('1.2.3.0/24')

In [ ]:
try:
    node1 = slice.get_node(name=node1_name)        
    node1_iface = node1.get_interface(network_name=network1_name)  
    
    print(f"Adding route, subnet: {partner_subnet}, gateway: {network1_gateway}") 
    node1.ip_route_add(subnet=partner_subnet, gateway=network1_gateway)
    
    print (f"{node1.get_name()} Routes:")
    stdout, stderr = node1.execute(f'ip route list')
except Exception as e:
    print(f"Exception: {e}")

## Test the routes you have configured

Try to ping other nodes owned by your partner.  Try to set and reset the routes to control which other users can access your VMs.


In [ ]:
try:
    partner_vm_ip = '<PARTNER_VM_IP>'
    
    node1 = slice.get_node(name=node1_name)        

    stdout, stderr = node1.execute(f'ping -c 5 {partner_vm_ip}')
    stdout, stderr = node1.execute(f'tracepath {partner_vm_ip}')
except Exception as e:
    print(f"Exception: {e}")

# Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Exception: {e}")